In [ ]:
%%px

comm.Barrier(); t1=MPI.Wtime()
for i in range(nx):
    if (i+rank*nx < Nx//2):
        X[i,:,:] = (i+rank*nx)*isotropic1024coarse['dx']
    else:
        X[i,:,:] = Lx-(i+rank*nx)*isotropic1024coarse['dx']

for j in range(ny):
    if (j < Ny//2):
        Y[:,j,:] = j*isotropic1024coarse['dy']
    else:
        Y[:,j,:] = Ly-j*isotropic1024coarse['dy']
    
for k in range(nz):
    if (k < Nz//2):
        Z[:,:,k] = k*isotropic1024coarse['dz']
    else:
        Z[:,:,k] = Lz-k*isotropic1024coarse['dz']
    
r2[:,:,:] = X[:,:,:]**2+Y[:,:,:]**2+Z[:,:,:]**2

r2rt = np.sqrt(r2)
comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Preparing the real domain for radial integration: {0:.2f} seconds\n'.format(t2-t1))

In [ ]:
%%px 

t = 5
c = 'r'

hist = []
r2st = []
    
#######################################

comm.Barrier(); t1=MPI.Wtime()

chi[:,:,:] = 0
chi[w2 > t*avgOmega] = 1

#######################################

comm.Barrier(); t2=MPI.Wtime()
cchi[:,:,:] = fft.forward3Dfft(chi,nx,ny,nz,nproc,rank)
    
tmp = cchi*(cchi.conj())
corr[:,:,:] = fft.backward3Dfft(tmp,nx,ny,nz,nproc,rank)
    
#######################################
    
comm.Barrier(); t3=MPI.Wtime()
        
#######################################
    
comm.Barrier(); t4=MPI.Wtime()
    
corrLoc,redges = np.histogram(r2rt,range=(0.5*dx,(ner+0.5)*dx),bins = rbins,weights=corr)
    
r2Loc,r2edges = np.histogram(r2rt,range=(0.5*dx,(ner+0.5)*dx),bins = rbins)
       
#######################################
    
comm.Barrier(); t5=MPI.Wtime()
    
corrLoc=np.float32(corrLoc)
comm.Reduce([corrLoc,MPI.REAL],[corrSum,MPI.REAL],op=MPI.SUM)
np.copyto(corrF,corrSum)
    
corrApend=np.zeros(ner,dtype='float32') 
corrApend[:] = corrF[:]
    
#########################
    
r2Loc=np.float32(r2Loc)
comm.Reduce([r2Loc,MPI.REAL],[r2Sum,MPI.REAL],op=MPI.SUM)
np.copyto(r2F,r2Sum)
    
r2Apend=np.zeros(ner,dtype='float32') 
r2Apend[:] = r2F[:]
    
if rank==0:
    hist.append(corrApend[:])
    r2st.append(r2Apend[:])
    
#######################################
    
comm.Barrier(); t6=MPI.Wtime()
    
if rank==0:
    sys.stdout.write('threshold used: {0:.2f} \n'.format(t))
    sys.stdout.write('Creating the excursion set: {0:.2f} seconds\n'.format(t2-t1))
    sys.stdout.write('Computing the 2-point correlation function: {0:.2f} seconds\n'.format(t3-t2))
    sys.stdout.write('Cleaning all but the 1st octant: {0:.2f} seconds\n'.format(t4-t3))
    sys.stdout.write('Integrating radially on each slab: {0:.2f} seconds\n'.format(t5-t4))
    sys.stdout.write('Adding the results form different slabs: {0:.2f} seconds\n'.format(t6-t5))
    sys.stdout.write('Full radial 2-point correlation calculation: {0:.2f} seconds\n'.format(t6-t1))
    sys.stdout.write('\n')